In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import random

In [3]:
city = 'irvine'
state = 'ca'
url = f'https://www.compass.com/homes-for-sale/{(city)}-{(state)}/'
page = requests.get(url)
soup = bs(page.content, 'html.parser')

In [87]:
times = [4,5,6,7,]
police_df = pd.read_csv('Data/police_df.csv')
soup_bowl = []
state = 'CA'
for city in police_df['City']:
    city_clean = city.replace(' ','-')
    url = f'https://www.compass.com/homes-for-sale/{city_clean.lower()}-{state.lower()}/'
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    soup_bowl.append(soup)
    time.sleep(random.choice(times))
len(soup_bowl)

34

In [88]:
price_list = []
for soup in soup_bowl:
    prices = soup.find_all('div', class_='uc-listingCard-mainStats')
    for price in prices:
        clean_price = price.text.replace('$','')
        clean_price = clean_price.replace(',','')
        price_list.append(int(clean_price))

In [90]:
#They placed a div tag inside a div tag for some unknown reason. Calling the 'text' method
#does not separate the number of beds from the word 'beds' with a space. This is my work around.

bed_list = []
bath_list = []
sqft_list = []
for soup in soup_bowl:
    beds = soup.find_all('div', class_='uc-listingCard-subStat uc-listingCard-subStat--beds')
    baths = soup.find_all('div', class_='uc-listingCard-subStat uc-listingCard-subStat--baths')
    sqfts = soup.find_all('div', class_='uc-listingCard-subStat uc-listingCard-subStat--sqFt')

    for i in range(len(beds)):
        bed_list.append(str(beds[i]).split('<')[1].split('>')[1])
        bath_list.append(str(baths[i]).split('<')[1].split('>')[1])
        sqft = str(sqfts[i]).split('<')[1].split('>')[1]
        if ',' in sqft:
            sqft_list.append(float(sqft.replace(',', '')))
        else:
            sqft_list.append(str(sqft))

In [104]:
cities = []
streets = []
zipcodes = []
for i,soup in enumerate(soup_bowl):
    addresses = soup.find_all('h2', class_='uc-listingCard-title truncate-overflow-text')
    for address in addresses:
        city = police_df['City'][i]
        street = str(address).split('target="_blank">')[1].split('</a>')[0]
        zipcode = str(address).split(', CA ')[1].split('</h2>')[0]
        cities.append(city)
        streets.append(street)
        zipcodes.append(zipcode)


In [105]:
Listings = pd.DataFrame({
    'Street':streets,    
    'City':cities,
    'Zipcode':zipcodes,
    'Listing Price ($)':price_list,
    'Number of Beds':bed_list,
    'Number of Baths':bath_list,
    'Square Footage': sqft_list
})

Listings['House ID'] = [i for i in range(len(Listings))]
Listings = Listings.set_index('House ID')

In [161]:
Clean_Listings = pd.DataFrame([])
for i in range(len(Listings)):
    try:
        float(Listings[Listings.index == i]['Number of Baths'][i])
        float(Listings[Listings.index == i]['Number of Beds'][i])
        float(Listings[Listings.index == i]['Square Footage'][i])
        Clean_Listings = Clean_Listings.append(Listings[Listings.index == i])
    except:
        continue
        

In [162]:
Clean_Listings['Square Footage'] = [float(item) for item in Clean_Listings['Square Footage']]

# Apparently one of the sqft values is a NaN. This row is dropped below
Clean_Listings = Clean_Listings[Clean_Listings['Square Footage'] >= 0]

In [163]:
Clean_Listings['House_ID'] = [i for i in range(len(Clean_Listings))]
Clean_Listings = Clean_Listings.set_index('House_ID')
#Clean_Listings.to_csv('Data/Home_Listings.csv')

In [164]:
Property_Price = Clean_Listings[['Listing Price ($)']]
Property_Amenities = Clean_Listings[['Number of Beds','Number of Baths','Square Footage']]
Property_Address = Clean_Listings[['Street','City','Zipcode']]
Property_Address['State'] = ['CA' for i in range(len(Property_Address))]

In [165]:
Property_Price.columns = ['Price']
Property_Address.columns = ['Street','City','Zipcode']
Property_Amenities.columns = ['Number_of_Beds','Number_of_Baths','Square_Footage']

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [180]:
# Create engine, declare base, reflect, show keys
engine = create_engine("sqlite:///SQL_db/NeighborhoodWatch.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['Police_Department',
 'Reported_Crime_Frequency',
 'Property_Address',
 'Property_Amenities',
 'Property_Price']

In [181]:
Property_Price.to_sql('Property_Price', con=engine, if_exists='append')
engine.execute("SELECT * FROM Property_Price").fetchall()

[(0, 595000),
 (1, 850000),
 (2, 899000),
 (3, 3795000),
 (4, 799900),
 (5, 1700000),
 (6, 699000),
 (7, 577000),
 (8, 675000),
 (9, 699900),
 (10, 850000),
 (11, 749900),
 (12, 830000),
 (13, 749000),
 (14, 699900),
 (15, 224900),
 (16, 754300),
 (17, 1250000),
 (18, 724900),
 (19, 1195000),
 (20, 1018000),
 (21, 769800),
 (22, 775000),
 (23, 975000),
 (24, 775000),
 (25, 824000),
 (26, 799000),
 (27, 1050000),
 (28, 550888),
 (29, 1650000),
 (30, 640000),
 (31, 317500),
 (32, 317500),
 (33, 1295000),
 (34, 459999),
 (35, 699900),
 (36, 659000),
 (37, 1099000),
 (38, 1500000),
 (39, 690000),
 (40, 725000),
 (41, 779000),
 (42, 690000),
 (43, 799999),
 (44, 685000),
 (45, 1100000),
 (46, 810000),
 (47, 849000),
 (48, 729000),
 (49, 695000),
 (50, 955900),
 (51, 720000),
 (52, 650000),
 (53, 479999),
 (54, 899000),
 (55, 129000),
 (56, 1600000),
 (57, 89500),
 (58, 1097000),
 (59, 2400000),
 (60, 1749000),
 (61, 700000),
 (62, 1100000),
 (63, 899500),
 (64, 2099999),
 (65, 2099999),
 (6

In [182]:
Property_Amenities.to_sql('Property_Amenities', con=engine, if_exists='append')
engine.execute("SELECT * FROM Property_Amenities").fetchall()

[(0, 2, 2.0, 984.0),
 (1, 4, 3.0, 2565.0),
 (2, 4, 2.5, 2002.0),
 (3, 24, 14.0, 9036.0),
 (4, 3, 2.0, 1514.0),
 (5, 10, 6.0, 5042.0),
 (6, 3, 2.0, 1279.0),
 (7, 3, 3.0, 1181.0),
 (8, 4, 3.5, 1709.0),
 (9, 3, 2.0, 1483.0),
 (10, 3, 2.5, 1831.0),
 (11, 3, 2.0, 1245.0),
 (12, 3, 2.5, 2024.0),
 (13, 3, 2.5, 2312.0),
 (14, 4, 2.0, 1555.0),
 (15, 2, 1.0, 1056.0),
 (16, 2, 2.0, 1621.0),
 (17, 8, 4.0, 3950.0),
 (18, 3, 2.0, 1238.0),
 (19, 4, 3.0, 2463.0),
 (20, 4, 3.5, 2200.0),
 (21, 3, 2.0, 1317.0),
 (22, 3, 2.0, 1511.0),
 (23, 4, 2.25, 2526.0),
 (24, 3, 2.0, 1377.0),
 (25, 3, 2.0, 1691.0),
 (26, 3, 2.0, 1562.0),
 (27, 5, 4.0, 3645.0),
 (28, 4, 3.0, 1499.0),
 (29, 4, 3.5, 4267.0),
 (30, 2, 2.0, 1046.0),
 (31, 3, 2.0, 1500.0),
 (32, 3, 2.0, 1500.0),
 (33, 4, 3.0, 2768.0),
 (34, 2, 1.5, 1104.0),
 (35, 3, 2.25, 1498.0),
 (36, 3, 3.0, 1519.0),
 (37, 4, 2.25, 2241.0),
 (38, 4, 3.5, 3646.0),
 (39, 3, 2.0, 1512.0),
 (40, 3, 1.75, 1330.0),
 (41, 3, 1.75, 1195.0),
 (42, 3, 3.0, 1486.0),
 (43, 4, 2.5, 

In [183]:
Property_Address.to_sql('Property_Address', con=engine, if_exists='append')
engine.execute("SELECT * FROM Property_Address").fetchall()

[(0, '7945 East Viewrim Drive', 'Anaheim', 'CA', 92808),
 (1, '1650 South Inez Way', 'Anaheim', 'CA', 92802),
 (2, '1275 South Goldstone', 'Anaheim', 'CA', 92804),
 (3, '324 East Leatrice Lane', 'Anaheim', 'CA', 92802),
 (4, '8425 East Meadowridge Street', 'Anaheim', 'CA', 92808),
 (5, '1841 West Crestwood Lane', 'Anaheim', 'CA', 92804),
 (6, '1473 East Rosewood Avenue', 'Anaheim', 'CA', 92805),
 (7, '1051 South Volterra Way', 'Anaheim', 'CA', 92808),
 (8, '3325 West Rovigo Drive', 'Anaheim', 'CA', 92801),
 (9, '745 North Clementine Street', 'Anaheim', 'CA', 92805),
 (10, '5803 East Mountain Loop Trail, Unit 26', 'Anaheim', 'CA', 92807),
 (11, '1218 North Ralston Street', 'Anaheim', 'CA', 92801),
 (12, '2404 East Briarvale Avenue', 'Anaheim', 'CA', 92806),
 (13, '6516 East Paseo Diego', 'Anaheim', 'CA', 92807),
 (14, '1509 West Chevy Chase Drive', 'Anaheim', 'CA', 92801),
 (15, '235 South Beach Boulevard, Unit 23', 'Anaheim', 'CA', 92804),
 (16, '1151 North Voyager Lane', 'Anaheim', 'C

In [185]:
#Sanity Check
pd.read_sql('Property_Address', con=engine)

,House_ID,Street,City,State,Zipcode
0,0,7945 East Viewrim Drive,Anaheim,CA,92808
1,1,1650 South Inez Way,Anaheim,CA,92802
2,2,1275 South Goldstone,Anaheim,CA,92804
3,3,324 East Leatrice Lane,Anaheim,CA,92802
4,4,8425 East Meadowridge Street,Anaheim,CA,92808
...,...,...,...,...,...
594,594,156 Nightingale Drive,Aliso Viejo,CA,92656
595,595,38 Edmonton Place,Aliso Viejo,CA,92656
596,596,57 Promontory Park,Aliso Viejo,CA,92656
597,597,11 Coffeeberry,Aliso Viejo,CA,92656
